In [81]:
from lxml import etree
import os 
import cv2

In [26]:
xmlsPath = '/Users/sandeep/Downloads/Data5/boxlabel'
imagesPath = '/Users/sandeep/Downloads/Data5/Image'
images = os.listdir(imagesPath)
xmls = os.listdir(xmlsPath)
#filter imagess
images = [i for i in images if f'{i[0:-4]}.xml' in xmls]

In [91]:
def getbboxCoord(obj):
    bbox = obj[1]
    return [int(c.text) for c in bbox.getchildren()] 
def getCoord(result):
    return [getbboxCoord(obj) for obj in result] 
def getlabel(image):
    imageLabelPath = '/Users/sandeep/Downloads/Data5/boxlabel/' + image[0:-4]+'.xml'
    imageXMLFile = etree.parse(imageLabelPath)
    x,y = 0,0
    #Get x
    path = "/annotation/object"
    result = imageXMLFile.xpath(path)
    bboxs = getCoord(result)
    return bboxs

In [94]:
def getImagePath(image):
    return '/Users/sandeep/Downloads/Data5/Image' + '/' + image

def getLabel(image):
    c = image[0:-4].split(',')
    return [int(c[0]),int(c[1])]    
def loadImages():
    return [cv2.imread(getImagePath(i)) for i in images]
def loadLabels():
    return [getlabel(i) for i in images]

#Loading Test Images and Lables
imgs  = loadImages()
labels = loadLabels()
    

#IoU

In [99]:
def calc_iou(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])

    # compute the area of intersection rectangle
    interArea = abs(max((xB - xA, 0)) * max((yB - yA), 0))
    if interArea == 0:
        return 0
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = abs((boxA[2] - boxA[0]) * (boxA[3] - boxA[1]))
    boxBArea = abs((boxB[2] - boxB[0]) * (boxB[3] - boxB[1]))

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)

    # return the intersection over union value
    return iou

In [104]:
def calc_precision_recall(image_results):
    """Calculates precision and recall from the set of images
    Args:
        img_results (dict): dictionary formatted like:
            {
                'img_id1': {'true_pos': int, 'false_pos': int, 'false_neg': int},
                'img_id2': ...
                ...
            }
    Returns:
        tuple: of floats of (precision, recall)
    """
    true_positive=0
    false_positive=0
    false_negative=0
    for img_id, res in image_results.items():
        true_positive +=res['true_positive']
        false_positive += res['false_positive']
        false_negative += res['false_negative']
        try:
            precision = true_positive/(true_positive+ false_positive)
        except ZeroDivisionError:
            precision=0.0
        try:
            recall = true_positive/(true_positive + false_negative)
        except ZeroDivisionError:
            recall=0.0
    return (precision, recall)

In [95]:
def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.
    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.
    Returns:
        dict: true positives (int), false positives (int), false negatives (int)
    """
    all_pred_indices= range(len(pred_boxes))
    all_gt_indices=range(len(gt_boxes))
    if len(all_pred_indices)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    if len(all_gt_indices)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    
    gt_idx_thr=[]
    pred_idx_thr=[]
    ious=[]
    for ipb, pred_box in enumerate(pred_boxes):
        for igb, gt_box in enumerate(gt_boxes):
            iou= calc_iou(gt_box, pred_box)
            
            if iou >iou_thr:
                gt_idx_thr.append(igb)
                pred_idx_thr.append(ipb)
                ious.append(iou)
    iou_sort = np.argsort(ious)[::1]
    if len(iou_sort)==0:
        tp=0
        fp=0
        fn=0
        return {'true_positive':tp, 'false_positive':fp, 'false_negative':fn}
    else:
        gt_match_idx=[]
        pred_match_idx=[]
        for idx in iou_sort:
            gt_idx=gt_idx_thr[idx]
            pr_idx= pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if(gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp= len(gt_match_idx)
        fp= len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)
    return {'true_positive': tp, 'false_positive': fp, 'false_negative': fn}

In [ ]:
#get Precision and Recall for each image
# gt_boxes, pred_boxes, iou_thr = labels[0] , labels[0] , 0.5
# res = get_single_image_results(gt_boxes,pred_boxes,iou_thr)




In [107]:
#Get Precision and recall 
# i = {'1':res, '2':res}
# calc_precision_recall(i)

(1.0, 1.0)